In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import seaborn as sns

In [54]:
from project_fraud.lib import drop_many_missing_values

In [55]:
data = drop_many_missing_values()
data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
data.shape

(590540, 226)

In [57]:
print(data.isnull().sum().sum())

16088043


## Convert mail column

In [58]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}


##us_emails = ['gmail',  'net',  'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
for c in ['P_emaildomain']:
    data[c + '_bin'] = data[c].map(emails)
    data[c + '_suffix'] = data[c].map(lambda x: str(x).split('.')[-1])
    

   # df_train[c + '_suffix'] = df_train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    #df_test[c + '_suffix'] = df_test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')



changing Nan values on Unknown 

In [59]:
# create a list of numerical features
# create a list of categorical features

c = (data.dtypes == 'object')
n = (data.dtypes != 'object')
cat_id_cols = list(c[c].index)
num_id_cols = list(n[n].index) 

print(cat_id_cols, "\n")
print("number categorical identity features: ", len(cat_id_cols), "\n\n")
print(num_id_cols, "\n")
print("number numerical identity features: ", len(num_id_cols))

['ProductCD', 'card4', 'card6', 'P_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain_bin', 'P_emaildomain_suffix'] 

number categorical identity features:  15 


['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', '

# LIST OF FEATURES TO USE 

In [ ]:
data['TransactionID']  #no null  - n
data['card1'] #no null  -n
data['card2']   # 8933   -n
data['addr1']    #65706    -n
data['TransactionAmt'] #no nul -n
data['card5']  #4259  -n
data['D15']  #89113  -n
data['C13'] #no null -n
data['D2']  # 280797 -n
data['D10'] # 76022   -n
data['D4']# 168922   -n
data['P_emaildomain_bin'] #engineered by us  no null -c
data['P_emaildomain_suffix']#engineered by us  no null  -c
data['TransactionDT'] #no null  - n

# ENCODING CATEGORICAL FEATURES 
### We have two categorical features that we need to encode: 'P_emaildomain_suffix', 'P_emaildomain_bin'

# imports 

In [61]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn.linear_model import LogisticRegression

# Imputing missing values 

In [62]:
# Create X and y
X = data[['TransactionID','P_emaildomain_suffix','P_emaildomain_bin',
'card1','card2','addr1','TransactionAmt','card5','D15','C13','D2','D10','D4','TransactionDT']]
y = data['isFraud']

data[['P_emaildomain_bin']]= data[['P_emaildomain_bin']].fillna(value = "Unknown")

In [63]:
n = (X.dtypes != 'object')
num_cols = list(n[n].index)
medium_missing_num_cols = []
low_missing_num_cols =[]
for i in num_cols:
    percentage = data[i].isnull().sum() * 100 / len(data[i])
    if percentage < 15:
        low_missing_num_cols.append(i)
    elif percentage >= 15 and percentage <= 60:
        medium_missing_num_cols.append(i)
        
        
        
        
        models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('SVM', SVC()))
models.append(('XGB', XGBClassifier()))
models.append(('RF', RandomForestClassifier()))

#testing models


In [65]:
num_transformer_low = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
num_transformer_medium = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

#cat_transformer = Pipeline([
    #'imputer', SimpleImputer(strategy='constant', fill_value = "Unknown")
    #])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('one_hot', OneHotEncoder())
])
    
preprocessor = ColumnTransformer([
    ('low_num_imputer',num_transformer_low, low_missing_num_cols),
    ('medium_num_imputer', num_transformer_medium, medium_missing_num_cols),
    ('cat_transformer', cat_pipeline, ['P_emaildomain_suffix','P_emaildomain_bin'])],
    remainder='passthrough')

pd.DataFrame(preprocessor.fit_transform(X)).head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-1.732048,0.821695,-3.629961e-16,0.253000,-0.278167,-1.393802,-0.243806,-6.510898e-01,-1.577987,-0.768856,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.732042,-1.457558,2.646603e-01,0.357260,-0.443327,-2.367147,-0.243806,-7.273558e-01,-1.577986,-0.768856,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-1.732036,-1.068263,8.138473e-01,0.409390,-0.317889,-0.809796,-0.243806,-7.273558e-01,-1.577972,0.880014,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-1.732030,1.679858,1.305561e+00,1.931586,-0.355521,-2.002143,-0.058284,-2.345599e-01,-1.577965,-0.187826,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-1.732024,-1.102133,9.671088e-01,1.347730,-0.355521,-2.367147,-0.243806,8.336966e-17,-1.577964,-0.496662,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# Train Test Split 


In [ ]:
# Train Test Split using random_state=414
# (let's forget for the sake of this challenge that we are data-leaking a bit here, we should have done the split earlier)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=414)


👇 Combine the following steps in a Pipeline:
- Impute missing values with a KNNImputer
- Scale all the features with a MinMaxScaler
- Model a LogisticRegression with default parameters
- Use the scoring metric relevant for the task

# import

In [ ]:
# (optional) Create here an train/eval split within the train set itself.
# Some powerfull models (XGBOOST, Neural Network...) which are prone to overfitting on the traning set, needs "early stopping criteria", to avoid descending the gradient completely and avoid overfitting.
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train)
X_train_train_small, X_train_test_small, y_train_train_small, y_train_test_small = train_test_split(X_train_small, y_train_small)

In [ ]:
pipe.predict(test_data)

In [ ]:


# Preprocessing pipeline
pipe = Pipeline([
    ('imputer', SimpleImputer())
    ('scaler', MinMaxScaler()),
    ('model', LogisticRegression() )
])

# Grid search KNNImputer parameter n_neighbors
grid_search = GridSearchCV(
    pipe, 
    param_grid={
        'imputer__n_neighbors': [2,5,10]},
        cv=5,
    scoring="recall")

grid_search.fit(data.drop(columns="malignant"), data['malignant'])

grid_search.best_params_

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clfs = []
clfs.append(LogisticRegression())
clfs.append(SVC())
clfs.append(SVC())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(GradientBoostingClassifier())

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X_train, y_train)
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())


#### Cross-Validation and Hyper Parameters Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
pipeline.set_params(clf= SVC())
pipeline.steps

In [ ]:
cv_grid = GridSearchCV(pipeline, param_grid = {
    'clf__kernel' : ['linear', 'rbf'],
    'clf__C' : np.linspace(0.1,1.2,12)
})

cv_grid.fit(X_train, y_train)

In [ ]:
cv_grid.best_params_

In [ ]:
y_predict = cv_grid.predict(X_test)
accuracy = accuracy_score(y_test,y_predict)
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))

# another 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

model_pipeline = Pipeline(steps=[('get_outlet_binary_columns', OutletTypeEncoder()), 
                                 ('pre_processing',pre_process),
                                 ('random_forest', RandomForestRegressor(max_depth=10,random_state=2))
                                 ])
# fit the pipeline with the training data
model_pipeline.fit(train_x,train_y)

# predict target values on the training data
model_pipeline.predict(train_x)


ANOTHER EXAMPLE

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree

pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression(random_state=42))])

pipe_svm = Pipeline([('scl', StandardScaler()),
                     ('pca', PCA(n_components=2)),
                     ('clf', svm.SVC(random_state=42))])
pipe_dt = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', tree.DecisionTreeClassifier(random_state=42))])


# List of pipelines for ease of iteration
pipelines = [pipe_lr, pipe_svm, pipe_dt]
# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {0: 'Logistic Regression', 1: 'Support Vector Machine', 2: 'Decision Tree'}
# Fit the pipelines
for pipe in pipelines:
pipe.fit(X_train, y_train)
# Compare accuracies
for idx, val in enumerate(pipelines):
print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))
# Identify the most accurate model on test data
best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, val in enumerate(pipelines):
    if val.score(X_test, y_test) > best_acc:
        best_acc = val.score(X_test, y_test)
        best_pipe = val
        best_clf = idx
        print('Classifier with best recall: %s' % pipe_dict[best_clf])

        
        
        
        


# BASELINE 

In [ ]:
# create Basemodel: SGDClassifier Logistic Regression 

log_reg_model = LogisticRegression(class_weight='balanced')
cross_val_score(log_reg_model, X_train, y_train, cv=3, scoring='recall')

base_model = SGDClassifier(loss='log', alpha=0.5, class_weight='balanced')
cv_results_base_model = cross_validate(base_model, X_train, y_train, cv=5, n_jobs=1, scoring=['recall', 'f1_macro'])
cv_results_base_model['test_f1_macro'].mean()

In [ ]:
# Logistic Regression Model 

log_reg_model = LogisticRegression(class_weight='balanced')
cv_results_log_reg_model = cross_val_score(log_reg_model, X_train, y_train, cv=5, scoring=['recall', 'f1_macro'])

## KNN


In [ ]:
# Import model
from sklearn.neighbors import KNeighborsClassifier

# Instanciate the model
knn_model = KNeighborsClassifier(n_neighbors=10)

# Train the model on the Training data
knn_model.fit(X_train, y_train)

# Score the model on the Testing data
knn_model.score(X_test,y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(range(1,25),score,color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.title('Score vs. K Neighbors')
plt.xlabel('K')
plt.ylabel('Accuracy')

Loop over different values of K and record the model's score for each value.

In [ ]:
score = []

for k in range(1,25):
    
    # Instanciate the model
    knn_model = KNeighborsClassifier(n_neighbors = k)

    # Train the model on the scaled Training data
    knn_model.fit(X_train, y_train)

    # Append the score 
    score.append(knn_model.score(X_test,y_test))
    

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(range(1,25),score,color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.title('Score vs. K Neighbors')
plt.xlabel('K')
plt.ylabel('Accuracy')

to see which value of K performs best

In [ ]:
import numpy as np

np.argmax(score)+1 # +1

In [ ]:
max(score)

# OR 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

knn = KNeighborsClassifier()
cross_validate(knn, X_train_scaled, y_train, cv=5, scoring='roc_auc')["test_score"].mean()

## 3. Grid search

Use KNeighborsClassifier

👇 Grid search a KNN's hyperparameter k on the training data.
- Search k = [1,5,10,20]
- 5-fold cross validate
- Score with recall

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Instanciate model
model = KNeighborsClassifier()

# Hyperparameter Grid
k_grid = {'n_neighbors' : [1, 5,10,20]}

# Instanciate Grid Search
grid = GridSearchCV(model, k_grid, n_jobs=-1, scoring = 'roc_auc', cv = 5)

# Fit data to Grid Search
grid.fit(X_train_scaled, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

Extract the best model from the grid search and score its performance on the test set.

In [ ]:
from sklearn.metrics import roc_auc_score
model = grid.best_estimator_
roc_auc_score(model.predict(scaler.transform(X_test)),y_test)

# OR


# Random Search 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

model = KNeighborsClassifier()

search_space = {'n_neighbors': randint(1, 40), 'p': [1, 2]}

search = RandomizedSearchCV(model, param_distributions=search_space,
                            n_jobs=-1, scoring='roc_auc', cv=5, n_iter=10)

search.fit(X_train_scaled, y_train)

print(search.best_score_)
print(search.best_params_)

## SVM Classifier for non-linearly separable data

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

# Instanciate model
model = SVC()

# Hyperparameter search space
search_space = {
    'kernel': ['sigmoid'],
    'C': stats.uniform(0.01, 1000),
    'gamma': stats.loguniform(0.001,10),
    'coef0': stats.uniform(-5,5),
}

# Instanciate Random Search
rsearch = RandomizedSearchCV(
    model, search_space,
    n_jobs=-1, scoring='accuracy', cv=5, n_iter=1000, verbose=1)


rsearch.fit(X,y)

In [ ]:
print(rsearch.best_params_)
print(rsearch.best_score_)
best_svm = rsearch.best_estimator_.fit(X,y)
plot_decision_regions(X, y, classifier=best_svm)

In [ ]:
from sklearn.model_selection import cross_val_score
print('CROSS VALIDATED RESULT')
print('mean accuracy', cross_val_score(best_svm, X, y, cv=10).mean())
print('std', cross_val_score(rsearch.best_estimator_, X, y, cv=10).std())

# VIF 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
df = pd.DataFrame()
df["vif_index"] = [vif(Xp, i) for i in range(Xp.shape[1])]
df

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clfs = []
clfs.append(LogisticRegression())
clfs.append(SVC())
clfs.append(SVC())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(GradientBoostingClassifier())

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X_train, y_train)
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())
